<a href="https://colab.research.google.com/github/otanet/NLP_seminar_20201022/blob/main/%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E6%99%82%E4%BB%A3%E3%81%AE%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E5%89%8D%E5%87%A6%E7%90%86_(2)_Sentencepiece_Python_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentencepiece python モジュール

このノートブックでは、Sentencepiece Pythonモジュールの使用方法を解説します。

## インストールと準備

sentencepiece は、pip コマンド経由で簡単に導入することができます。

このノートブックでは、小説[坊っちゃん](https://en.wikipedia.org/wiki/Botchan) の英訳を入力データのサンプルとして用います。 同データは sentencepiece のパッケージの中に同封されています。

In [ ]:
!pip install sentencepiece
!wget https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt

--2020-10-19 16:15:02--  https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278779 (272K) [text/plain]
Saving to: ‘botchan.txt.1’

botchan.txt.1       100%[===================>] 272.25K  --.-KB/s    in 0.03s   

2020-10-19 16:15:03 (8.64 MB/s) - ‘botchan.txt.1’ saved [278779/278779]



## 基本的な動作方法

###  サブワードの学習

サブワードの学習は、**SentencePieceTrainer.train** static メソッドを用います。コマンドラインツール spm_train の引数を python の引数スタイルで与えます。

In [ ]:
import sentencepiece as spm

# 'botchan.txt' を用い学習し、'm.model', 'm.vocab' を生成
# 'm.vocab' はデバッグ目的に出力され、分割時には不要
spm.SentencePieceTrainer.train(input='botchan.txt', model_prefix='m', vocab_size=2000)

### トークン化 (エンコード)

トークン化には、**SentencePieceProcessor** クラスを用います。コンストラクタにモデルファイルを渡し初期化します。

**encode** メソッドでトークン化を行います。**out_type** を指定することで、トークン文字列あるいは id 列の出力を切り替えることができます。**encode** は文のリストも受け付けます。この場合は、結果のリストが返ります。

In [ ]:
sp = spm.SentencePieceProcessor(model_file='m.model')

# encode: text => トークン列 あるいは id 列への変換
print(sp.encode('This is a test', out_type=str))
print(sp.encode('This is a test', out_type=int))

# 複数文の処理
print(sp.encode(['This is a test', 'hello world', 'I saw a girl with a telescope'],
                out_type=int))

['▁This', '▁is', '▁a', '▁t', 'est']
[212, 32, 10, 587, 446]
[[212, 32, 10, 587, 446], [29, 134, 44, 1040], [6, 291, 89, 10, 1097, 26, 10, 9, 228, 126, 8, 82, 310, 20]]


### 脱トークン化 (デコード)

**decode** メソッドを用いて、id 列やトークン列から元の文を復元します。**encode** と同様、リスト入力を受け付けます。

In [ ]:
# decode: id => text
print(sp.decode(['▁This', '▁is', '▁a', '▁t', 'est']))
print(sp.decode([212, 32, 10, 587, 446]))
print(sp.decode([[212, 32, 10, 587, 446],[29, 134, 44, 1040]]))

This is a test
This is a test
['This is a test', 'hello world']


### トークンとIDのマッピング

**piece_size** を使い語彙数を取得することができます。

**id_to_piece**, **piece_to_id** を用い、id <=> トークンの変換を行います。
**is_control**は、 id が制御トークンかを返します。同様に、**is_unknown** は、未知文字かどうかを返します。

これらのメソッドも同様に、リストを受け付けます。

In [ ]:
# 語彙サイズ
print(sp.piece_size())

# id <=> piece 変換
print(sp.id_to_piece(209))
print(sp.piece_to_id('▁This'))
print(sp.piece_to_id(['▁This', '▁is', '▁a', '▁t', 'est']))
print(sp.id_to_piece([212, 32, 10, 587, 446]))

# 未知語は id 0 が返る (変更可能)
print(sp.piece_to_id('__MUST_BE_UNKNOWN__'))

# <unk>, <s>, </s> の id の表示
print(sp.id_to_piece([0,1,2]))
print(sp.is_control([0,1,2]))
print(sp.is_unknown([0,1,2]))

2000
ok
212
[212, 32, 10, 587, 446]
['▁This', '▁is', '▁a', '▁t', 'est']
0
['<unk>', '<s>', '</s>']
[False, True, True]
[True, False, False]


## 実ファイルに依存しないオンメモリ処理

Sentencepiece の学習およびトークン化は、物理的なファイルに依存することなく、メモリ上で行うことが可能です。



### ファイルに依存しない学習
**SentencePieceTrainer.train()** は、**sentence_iterator** 引数に任意のイテラブルなオブジェクトを渡すことで、文の集合をイテレーターとしてフィードすることができます。
さらに、**model_writer** に write メソッドを持つ任意のオブジェクトを渡すことで、
モデルのbyte表現の書き込みを移譲することができます。

以下は、文が格納された文字列 list から学習し、BytesIO にモデルを書き込む例です。

In [ ]:
import io

train = []
with open('botchan.txt', 'r') as f:
    for line in f:
        line = line.rstrip()
        train.append(line)

model_proto = io.BytesIO()

spm.SentencePieceTrainer.train(sentence_iterator=iter(train), 
                                model_writer=model_proto,
                               vocab_size=2000)

with open('spm_model.model', 'wb') as f:
    f.write(model_proto.getvalue())

イテラブルなオブジェクトであれば、どのような入力からも学習できます。以下は、外部サイトファイルから直接学習する例です。

In [ ]:
import urllib.request
import io
import sentencepiece as spm

model = io.BytesIO()
with urllib.request.urlopen(
    'https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt'
) as response:
  spm.SentencePieceTrainer.train(
      sentence_iterator=response, model_writer=model, vocab_size=2000)
  
  sp = spm.SentencePieceProcessor(model_proto=model.getvalue())
  print(sp.encode('hello world', out_type=str))

['▁he', 'll', 'o', '▁world']


### ファイルに依存しないトークン化

**model_proto** パラメータにモデルのバイト列を渡すことで、メモリ上のモデルから **SentencepieceProcessor** を初期化できます。

In [ ]:
sp = spm.SentencePieceProcessor(model_proto=model_proto.getvalue())

print(sp.encode('This is a test'))

[212, 32, 10, 587, 446]


**serialized_model_proto** メソッドを呼ぶことで、モデルのバイト表現を取得できます。

In [ ]:
model_proto2 = sp.serialized_model_proto()

sp2 = spm.SentencePieceProcessor(model_proto=model_proto2)
print(sp2.encode('This is a test'))

[212, 32, 10, 587, 446]


### Pickle オブジェクト

Sentenceiece は、python 標準の pickle 経由でシリアライズ・デシリアライズが可能です。

In [ ]:
import pickle

sp = spm.SentencePieceProcessor(model_proto=model_proto.getvalue())
print(sp.encode('This is a test'))

with open('spm.pickle', 'wb') as f:
  pickle.dump(sp, f)

with open('spm.pickle', 'rb') as f:
  sp2 = pickle.load(f)

print(sp2.encode('This is a test'))

[212, 32, 10, 587, 446]
[212, 32, 10, 587, 446]


## BOS, EOS, UNK, PAD トークン

### BOS, EOS, PAD トークンの表示
**bos_id, eos_id, unk_id, pad_id** を用いることで、それぞれの特殊トークンのid を取得できます。**pad_id** はデフォルトでは未定義 (-1) です。

In [ ]:
print(sp.unk_id())
print(sp.bos_id())
print(sp.eos_id())
print(sp.pad_id())

0
1
2
-1


### BOS, EOS の自動挿入
encode 時に, add_bos, add_eos を指定することで自動的に bos/eos トークンを挿入します。reverse を指定すると入力を反転させます。

In [ ]:
print(sp.encode('This is a test'))

print(sp.encode('This is a test', add_bos=True, add_eos=True))

print(sp.encode('This is a test', reverse=True))

[212, 32, 10, 587, 446]
[1, 212, 32, 10, 587, 446, 2]
[446, 587, 10, 32, 212]


### BOS, EOS, UNK, PAD トークンの書き換え

学習時に **pad_id, unk_id, bos_id, eos_id** 引数を使い、それぞれの id を上書きできます。id に -1 が指定されると、その特殊トークンを無効化します。同様に、**pad_piece, bos_piece, eos_iece, unk_piece** を指定することで、内部の文字列表現を書き換えることができます。

In [ ]:
spm.SentencePieceTrainer.train(input='botchan.txt',
                                vocab_size=2000,
                                model_prefix='m',
                                pad_id=0,
                                unk_id=1,
                                bos_id=2,
                                eos_id=-1,
                                pad_piece="[PAD]",
                                unk_piece="[UNK]",
                                bos_piece="[BOS]")

sp = spm.SentencePieceProcessor(model_file='m.model')

print(sp.unk_id())
print(sp.bos_id())
print(sp.eos_id())
print(sp.pad_id())

print(sp.id_to_piece([0,1,2]))


1
2
-1
0
['[PAD]', '[UNK]', '[BOS]']


## UTF8 バイトフォールバック

**byte_fallbacl=True** を指定することで、未知文字を utf8 文字に分解する byte_fallback を有効にすることができます。 byte_fallback を有効にするときには、**character_coverage** を 1未満にし、フォールバックされたトークンが学習中に出現するようにすることをおすすめします。

In [ ]:
spm.SentencePieceTrainer.train(input='botchan.txt',
                                vocab_size=2000,
                                model_prefix='m', byte_fallback=True)

sp = spm.SentencePieceProcessor(model_file='m.model')

# 😀 は未知文字。学習データに出現しない
pieces = sp.encode('hello world.😀', out_type=str)
ids = sp.encode('hello world.😀', out_type=int)

# <0xXX> トークンとして出力
print(pieces)
print(ids)

# バイト列から utf8 に戻す
print(sp.decode(pieces))
print(sp.decode(ids))

['▁he', 'll', 'o', '▁world', '.', '<0xF0>', '<0x9F>', '<0x98>', '<0x80>']
[285, 390, 300, 1296, 260, 243, 162, 155, 131]
hello world.😀
hello world.😀


## 独自シンボルの定義

ユーザ独自のシンボルを定義することができます。例として、
[BERT](https://arxiv.org/abs/1810.04805)の [SEP] や [CLS] といった特殊シンボルが挙げられます。 

独自シンボルには以下の2種類があります。

- **ユーザ定義シンボル**: どのような文脈でも1トークンとして切り出されます。ユーザ辞書のような働きをします。このトークンは入力文字列に出現することができます。
- **制御シンボル**:  制御用にid を予約します。制御シンボルは、入力に出現することを想定しておらず、出現しても通常の分割が行われます。ユーザは、予約されたIDをタスクに応じて追加、挿入します。

### ユーザ定義シンボルの例

ユーザ定義シンボルは  **user_defined_symbols** パラメータを使いリスト形式で指定します。

In [ ]:
spm.SentencePieceTrainer.train(input='botchan.txt',
                               model_prefix='m_user',
                              user_defined_symbols=['<sep>','<cls>'], vocab_size=2000)

sp_user = spm.SentencePieceProcessor(model_file='m_user.model')

# <unk>=0, <s>=1, </s>=2, <sep>=3, <cls>=4
# ユーザ定義シンボルは、ユーザ辞書のように振る舞い、入力文字列に出現したときには
# その単位で切り出されます。
print(sp_user.encode('this is a test<sep> hello world<cls>', out_type=str))
print(sp_user.encode('this is a test<sep> hello world<cls>', out_type=int))
print(sp_user.piece_to_id('<sep>'))  # 3
print(sp_user.piece_to_id('<cls>'))  # 4
print('3=', sp_user.decode_ids([3]))  # decoded to <sep>
print('4=', sp_user.decode_ids([4]))  # decoded to <cls>

['▁this', '▁is', '▁a', '▁t', 'est', '<sep>', '▁he', 'll', 'o', '▁world', '<cls>']
[48, 34, 12, 589, 448, 3, 31, 136, 46, 1042, 4]
3
4
3= <sep>
4= <cls>


### 制御シンボルの例

制御シンボルは  **control_symbols** パラメータを使いリスト形式で指定します。

In [ ]:
spm.SentencePieceTrainer.train(input='botchan.txt',
                               model_prefix='m_ctrl',
                               control_symbols=['<sep>','<cls>'], vocab_size=2000)

sp_ctrl = spm.SentencePieceProcessor(model_file='m_ctrl.model')

# 制御シンボルは id のみが予約されます。
print(sp_ctrl.encode('this is a test<sep> hello world<cls>', out_type=str))
print(sp_ctrl.encode('this is a test<sep> hello world<cls>', out_type=int))
print(sp_ctrl.piece_to_id('<sep>'))  # 3
print(sp_ctrl.piece_to_id('<cls>'))  # 4
print('3=', sp_ctrl.decode_ids([3]))  # decoded to empty
print('4=', sp_ctrl.decode_ids([4]))  # decoded to empty

['▁this', '▁is', '▁a', '▁t', 'est', '<', 'se', 'p', '>', '▁he', 'll', 'o', '▁world', '<', 'c', 'l', 's', '>']
[48, 34, 12, 589, 448, 0, 95, 78, 0, 31, 136, 46, 1042, 0, 84, 60, 10, 0]
3
4
3= 
4= 


BOS/EOS (&lt;s&gt;, &lt;/s&gt;) は、制御シンボルとして定義されています。これらをユーザ定義シンボルとして再定義することで、入力テキストにこれらのシンボルが含まれると BOS, EOS の動作をするようになります。

In [ ]:
spm.SentencePieceTrainer.train(input='botchan.txt',
                               model_prefix='m',
                               vocab_size=2000)

spm.SentencePieceTrainer.train(input='botchan.txt',
                               model_prefix='m_bos_as_user',
                               user_defined_symbols=['<s>','</s>'],
                               vocab_size=2000)

# デフォルト動作: <s>, </s> は制御シンボル。入力に含まれていても
# 通常通り分割される
sp = spm.SentencePieceProcessor(model_file='m.model')
print(sp.encode('<s> hello</s>', out_type=str))
print(sp.encode('<s> hello</s>', out_type=int))

sp = spm.SentencePieceProcessor(model_file='m_bos_as_user.model')
# <s>, </s> をユーザシンボルとして再定義。入力に含まれると、BOS/EOSとして扱われる。
print(sp.encode('<s> hello</s>', out_type=str))
print(sp.encode('<s> hello</s>', out_type=int))

['▁', '<', 's', '>', '▁he', 'll', 'o', '</', 's', '>']
[9, 0, 8, 0, 29, 134, 44, 0, 8, 0]
['▁', '<s>', '▁he', 'll', 'o', '</s>']
[9, 1, 29, 134, 44, 2]


## サブワード正則化とn-best出力

### サブワード正則化
**encode** メソッドにて **enable_sampling=True** を指定することで、メソッドの呼び出し毎に確率的に分割の異なる結果を返します。**alpha** パラメータで、分布の偏りを変更できます。小さい値ほど、多様な解を出しやすくなります。

**nbest_size** が指定されると、nbest からサンプリングを行います。探索空間が限定されることで、最適解に近い分割に制限しやすくなります。nbest_size は、model_type が unigram のときのみ有効です。

In [ ]:
print('alpha=0.5')
for n in range(10):
  print(sp.encode('hello world', out_type=str, enable_sampling=True, alpha=0.5))

print('\nalpha=0.01')
for n in range(10):
  print(sp.encode('hello world', out_type=str, enable_sampling=True, alpha=0.01))

print('\n alpha=0.01, nbest=3')
for n in range(10):
  print(sp.encode('hello world', out_type=str, enable_sampling=True, alpha=0.01, nbest_size=3))

alpha=0.5
['▁he', 'll', 'o', '▁wor', 'ld']
['▁he', 'll', 'o', '▁world']
['▁he', 'll', 'o', '▁world']
['▁he', 'll', 'o', '▁world']
['▁he', 'll', 'o', '▁world']
['▁he', 'll', 'o', '▁world']
['▁he', 'll', 'o', '▁world']
['▁he', 'll', 'o', '▁world']
['▁he', 'll', 'o', '▁world']
['▁he', 'll', 'o', '▁world']

alpha=0.01
['▁', 'h', 'e', 'l', 'l', 'o', '▁w', 'or', 'ld']
['▁', 'h', 'e', 'l', 'l', 'o', '▁', 'w', 'o', 'r', 'ld']
['▁', 'he', 'll', 'o', '▁wor', 'ld']
['▁he', 'l', 'l', 'o', '▁w', 'or', 'l', 'd']
['▁', 'h', 'e', 'll', 'o', '▁', 'w', 'o', 'r', 'l', 'd']
['▁he', 'l', 'l', 'o', '▁', 'wo', 'r', 'ld']
['▁', 'h', 'e', 'l', 'l', 'o', '▁w', 'or', 'ld']
['▁', 'he', 'l', 'l', 'o', '▁w', 'or', 'ld']
['▁', 'h', 'el', 'l', 'o', '▁', 'w', 'or', 'l', 'd']
['▁', 'he', 'll', 'o', '▁', 'wo', 'r', 'ld']

 alpha=0.01, nbest=3
['▁', 'he', 'll', 'o', '▁world']
['▁he', 'll', 'o', '▁world']
['▁he', 'l', 'l', 'o', '▁world']
['▁he', 'll', 'o', '▁world']
['▁', 'he', 'll', 'o', '▁world']
['▁he', 'll', 'o', '▁wo

### nbest 解の出力

**nbest_encode_as_ids**, **nbest_encode_as_piece** メソッドで nbest 解を取得することができます。

In [ ]:
print(sp.nbest_encode_as_ids('hello world', nbest_size=3))

[[29, 134, 44, 1040], [29, 58, 58, 44, 1040], [9, 787, 134, 44, 1040]]


# 様々な分割モデル

### BPE (Byte pair encoding) モデル

**model_type='bpe'** を指定することで、bpe 分割が有効になります。

In [ ]:
spm.SentencePieceTrainer.train(input='botchan.txt',
                                model_prefix='m_bpe',
                                vocab_size=2000,
                                model_type='bpe')
sp_bpe = spm.SentencePieceProcessor(model_file='m_bpe.model')

print(sp_bpe.encode('this is a test', out_type=str))

['▁this', '▁is', '▁a', '▁t', 'est']


### 文字・単語分割モデル

**model_type='word'**, **model_type='char'** を指定することで、それぞれ単語分割, 文字分割モデルが有効になります。

`word` モデルを用いるときは、入力テキストは事前に分かち書きされている必要があります。

In [ ]:
spm.SentencePieceTrainer.train(input='botchan.txt',
                              model_prefix='m_char', model_type='char',
                              vocab_size=400)

sp_char = spm.SentencePieceProcessor(model_file='m_char.model')

print(sp_char.encode('this is a test.', out_type=str))


['▁', 't', 'h', 'i', 's', '▁', 'i', 's', '▁', 'a', '▁', 't', 'e', 's', 't', '.']


In [ ]:
spm.SentencePieceTrainer.train(input='botchan.txt',
                              model_prefix='m_word', model_type='word',
                              vocab_size=200)

sp_word = spm.SentencePieceProcessor(model_file='m_word.model')

print(sp_word.encode('this is a test.', out_type=str))

['▁this', '▁is', '▁a', '▁test.']


## テキスト正規化

Sentencepiece は、入力文字列を Unicode NFCK で正規化します。ユーザは正規化ルールを変更したり、独自のルールを使って正規化することが可能です。正規化ルールはモデルファイルに埋め込まれます。同一モデルファイルを使う限り、同一の正規化が行われることが保証されれ、前処理の再現性が確保されます。

### 定義済みテキスト正規化の変更

Sentencepiece は、以下の定義済み的すすと正規化ルールを提供しています。 **normaliation_rule_name=&lt;NAME&gt;** 引数にて、正規化ルールを変更することができます。正規化ルールの情報は、モデルファイルに保存されるため、トークン化時に再指定する必要はありません。

- **nmt_nfkc**: Unicode NFKC ルール + 空白まわりの独自ルール (デフォルト)
- **nfkc: original**: Unicode NFKC ルール
- **nmt_nfkc_cf**: nmt_nfkc + 小文字化
- **nfkc_cf**:  nfkc + 小文字化
- **identity**: 正規化なし

In [ ]:
import sentencepiece as spm

# NFKC+小文字化
spm.SentencePieceTrainer.train(input='botchan.txt',
                              model_prefix='m_nfkc_cf', vocab_size=2000,
                              normalization_rule_name='nfkc_cf')

sp = spm.SentencePieceProcessor(model_file='m_nfkc_cf.model')

# 小文字化
print(sp.encode('ＨＥＬＬＯ　ＷＯＲＬＤ.', out_type=str)) 

['▁', 'hello', '▁world', '.']


### 独自正規化ルールの作成

正規化ルールは、文字列から文字列への書き換え規則として実装されています。書き換えに曖昧性があるばあいは、最長のルールが用いられます。ユーザ定義の正規化ルールはTSVファイルとして与えます。定義済みルールのTSVファイルはdataディレクトリあります([サンプル](https://raw.githubusercontent.com/google/sentencepiece/master/data/nfkc.tsv))。正規化ルールは、FSTにコンパイルされモデルファイルに埋め込まれます。そのため、トークン化の際 TSVファイルを再指定する必要はありません。
TSVファイルは、学習時に **normalization_rule_tsv=&lt;FILE&gt;**パラメータとして与えます。

In [ ]:
def tocode(s):                                                                               
    out = [str(hex(ord(c))).replace('0x', 'U+') for c in s]                                                                                                                      
    return ' '.join(out)          

# TSV format:  書換元 Unicode コードポイント 列 <tab> 書き換え先コードポイント列
# 例:  normalize "don't => do not,  I'm => I am"
with open('normalization_rule.tsv', 'w') as f:
  f.write(tocode("I'm") + '\t' + tocode("I am") + '\n')
  f.write(tocode("don't") + '\t' + tocode("do not") + '\n')

print(open('normalization_rule.tsv', 'r').read())

# 正規化ルールファイルを指定して学習
spm.SentencePieceTrainer.train(input='botchan.txt',
                               model_prefix='m_custom',
                              vocab_size=2000,
                                normalization_rule_tsv='normalization_rule.tsv')

sp = spm.SentencePieceProcessor(model_file='m_custom.model')

print(sp.encode("I'm busy", out_type=str))  # normalzied to `I am busy'
print(sp.encode("I don't know it.", out_type=str))  # normalized to 'I do not know it.'

U+49 U+27 U+6d	U+49 U+20 U+61 U+6d
U+64 U+6f U+6e U+27 U+74	U+64 U+6f U+20 U+6e U+6f U+74

['▁I', '▁am', '▁bu', 's', 'y']
['▁I', '▁do', '▁not', '▁know', '▁it', '.']


## 語彙の制限

**set_vocabulary** にトークン集合を渡すことで、encode で使われるトークンを制限することができます。指定された語彙集合のみを使ってトークン化を行います。同制限の詳細については、
 [subword-nmt page](https://github.com/rsennrich/subword-nmt#best-practice-advice-for-byte-pair-encoding-in-nmt) を御覧ください。

In [ ]:
# すべてのトークンを Python list として取得
vocabs = [sp.id_to_piece(id) for id in range(sp.get_piece_size())]

# 各トークンの出現頻度を計算
freq = {}
with open('botchan.txt', 'r') as f:
    for line in f:
        line = line.rstrip()
        for piece in sp.encode(line, out_type=str):
            freq.setdefault(piece, 0)
            freq[piece] += 1
          
# 1000回以上出現した部分集合を作成
vocabs = list(filter(lambda x : x in freq and freq[x] > 1000, vocabs))

# 語彙制限前の分割
print(sp.encode('this is a test.', out_type=str))

# 語彙制限の有効化
sp.set_vocabulary(vocabs)
print(sp.encode('this is a test.', out_type=str))

# 語彙制限をリセット
sp.reset_vocabulary()
print(sp.encode('this is a test.', out_type=str))

['▁this', '▁is', '▁a', '▁t', 'est', '.']
['▁', 't', 'h', 'i', 's', '▁', 'i', 's', '▁a', '▁', 't', 'e', 's', 't', '.']
['▁this', '▁is', '▁a', '▁t', 'est', '.']


## 複数単語にまたがるトークンの学習

デフォルト動作では、空白をトークンの区切り制約とするため、複数の単語(ここでの単語とは、空白で区切られたトークンを意味する)にまたがるトークンを抽出することはありません。

**split_by_whtespace=False** を指定して学習することで、この動作を停止し、複数の単語にまたがるトークンを抽出するようになります。ただし、中国語や日本語の場合は、文が空白で区切られていないため、このパラメータによって動作が大きく変わることはないでしょう。

In [ ]:
import re

spm.SentencePieceTrainer.train(input='botchan.txt',
                                model_prefix='m_cross_word',
                                vocab_size=2000,
                                split_by_whitespace=False)

sp = spm.SentencePieceProcessor(model_file='m_cross_word.model')

# 学習されたトークンをPython list として取得
vocabs = [sp.id_to_piece(id) for id in range(sp.piece_size())]

# 複数単語にまたがるトークンを出力
for piece in vocabs[0:500]:
    if re.match('\w+▁\w+', piece):
        print(piece)

ed▁to
roject▁Gutenberg
s▁of
ing▁the
ed▁me
ed▁the
ed▁in


## 頻度付き単語集合からの学習

Sentencepiece は、頻度付きの単語集合からのサブワード学習をサポートしています。

学習データとして、1カラム目が単語、2カラム目が頻度となるようなTSVファイルを作成します。
学習時に、**input_format='tsv'** を指定することで、入力ファイルのフォーマットをTSVに変更します。 TSVファイルが入力の場合は、**split_by_whtespace=true** が設定されているもととみなします。


In [ ]:
freq={}
with open('botchan.txt', 'r') as f:
  for line in f:
    line = line.rstrip()
    for piece in line.split():
      freq.setdefault(piece, 0)
      freq[piece] += 1
            
with open('word_freq_list.tsv', 'w') as f:
  for k, v in freq.items():
    f.write('%s\t%d\n' % (k, v))
  
spm.SentencePieceTrainer.train(input='word_freq_list.tsv',
                               input_format='tsv',
                               model_prefix='m_word_freq',
                               vocab_size=2000)
sp = spm.SentencePieceProcessor(model_file='m_word_freq.model')

print(sp.encode('this is a test.', out_type=str))

['▁this', '▁is', '▁a', '▁t', 'est', '.']


## トークンの文字オフセットの取得

Sentencepiece は、各トークンが元の入力文のどの位置に対応するか、utf8 byte 単位でオフセットを保持しています。オフセット情報は、文字列のハイライト等に利用可能です。

オフセット情報を利用するには、protobuf モジュールと sentencepiece_pb2.py が必要となります。SentencePieceProcessor の **encode_as_serialized_proto** 経由で、シリアライズされたSentencePieceText proto が取得できます。同バイト列をデシリアライズし、SentencePieceText proto に変換します。SentencePieceText は、オフセット以外にも、トークンの文字列表現、ID等が保持されています。

SentencePieceText のproto 定義は [こちら](https://github.com/google/sentencepiece/blob/3be3f2e11e2bb923c579c6be5e7335809341587f/src/sentencepiece.proto#L23)にあります。


In [ ]:
!pip install protobuf
!wget https://raw.githubusercontent.com/google/sentencepiece/master/python/sentencepiece_pb2.py

--2020-10-19 15:00:55--  https://raw.githubusercontent.com/google/sentencepiece/master/python/sentencepiece_pb2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7382 (7.2K) [text/plain]
Saving to: ‘sentencepiece_pb2.py’

sentencepiece_pb2.p 100%[===================>]   7.21K  --.-KB/s    in 0s      

2020-10-19 15:00:55 (83.8 MB/s) - ‘sentencepiece_pb2.py’ saved [7382/7382]



In [ ]:
import sentencepiece_pb2

spm.SentencePieceTrainer.train(input='botchan.txt',
                                model_prefix='m',
                                vocab_size=2000)

sp = spm.SentencePieceProcessor(model_file='m.model')

# One best result
spt = sentencepiece_pb2.SentencePieceText()
spt.ParseFromString(sp.encode_as_serialized_proto('ｈｅｌｌｏ')) # Full width hello

# begin/end (offsets) are pointing to the original input.
print('============== one best ================')
print(spt)

# Nbest results
nspt = sentencepiece_pb2.NBestSentencePieceText()
nspt.ParseFromString(sp.nbest_encode_as_serialized_proto('ｈｅｌｌｏ', 3))
print('============== nbest ================')
print(nspt)

============== one best ================
text: "\357\275\210\357\275\205\357\275\214\357\275\214\357\275\217"
pieces {
  piece: "\342\226\201he"
  id: 29
  surface: "\357\275\210\357\275\205"
  begin: 0
  end: 6
}
pieces {
  piece: "ll"
  id: 134
  surface: "\357\275\214\357\275\214"
  begin: 6
  end: 12
}
pieces {
  piece: "o"
  id: 44
  surface: "\357\275\217"
  begin: 12
  end: 15
}

============== nbest ================
nbests {
  text: "\357\275\210\357\275\205\357\275\214\357\275\214\357\275\217"
  pieces {
    piece: "\342\226\201he"
    id: 29
    surface: "\357\275\210\357\275\205"
    begin: 0
    end: 6
  }
  pieces {
    piece: "ll"
    id: 134
    surface: "\357\275\214\357\275\214"
    begin: 6
    end: 12
  }
  pieces {
    piece: "o"
    id: 44
    surface: "\357\275\217"
    begin: 12
    end: 15
  }
  score: -17.582029342651367
}
nbests {
  text: "\357\275\210\357\275\205\357\275\214\357\275\214\357\275\217"
  pieces {
    piece: "\342\226\201he"
    id: 29
    surfac